<a href="https://colab.research.google.com/github/jingkunzler211/Movie_Recommender/blob/main/JK_HW2_Movies_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spark HW2 Movie Recommendation**

In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in MovieLens small dataset: https://grouplens.org/datasets/movielens/latest/

# Data ETL and Data Exploration

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q http://apache.forsale.plus/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!ls
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!wget -q http://apache.mirrors.pair.com/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (125 kB/s)
Reading package lists... Done
drive			       spark-3.0.1-bin-hadoop2

In [ ]:
# set up spark

!pip install -q findspark
!pip install py4j

!export JAVA_HOME=$(/usr/lib/jvm/java-8-openjdk-amd64 -v 1.8)
! echo $JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

/bin/bash: /usr/lib/jvm/java-8-openjdk-amd64: Is a directory



In [ ]:
!ls

drive				 spark-3.0.1-bin-hadoop2.7.tgz.2
sample_data			 spark-3.0.1-bin-hadoop2.7.tgz.3
spark-3.0.1-bin-hadoop2.7	 spark-3.0.1-bin-hadoop2.7.tgz.4
spark-3.0.1-bin-hadoop2.7.tgz	 spark-3.0.1-bin-hadoop2.7.tgz.5
spark-3.0.1-bin-hadoop2.7.tgz.1


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
movies_df = spark.read.load("drive/My Drive/movie_rec_small/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("drive/My Drive/movie_rec_small/ratings.csv", format='csv', header = True)
links_df = spark.read.load("drive/My Drive/movie_rec_small/links.csv", format='csv', header = True)
tags_df = spark.read.load("drive/My Drive/movie_rec_small/tags.csv", format='csv', header = True)

In [ ]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [ ]:
links_df.show(5)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows



In [ ]:
tags_df.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [ ]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
tmp3 = ratings_df.groupBy("userID").count().toPandas()['count'].mean()
tmp4 = ratings_df.groupBy("movieId").count().toPandas()['count'].mean()

print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))
print('Average number of ratings per user is {}'.format(tmp1))
print('Average number of ratings per movie is {}'.format(tmp1))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1
Average number of ratings per user is 20
Average number of ratings per movie is 20


# Part 1: Spark SQL and OLAP

In [ ]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

**1. Number of users**

In [ ]:
q1_result=spark.sql("Select Count(Distinct userId) as Number_of_Users from ratings")
q1_result.show()

+---------------+
|Number_of_Users|
+---------------+
|            610|
+---------------+



**2. Number of Movies**
**and number of movies not rated**

In [ ]:
q2_result=spark.sql("Select Count(movieId) as Number_of_Movies from movies")
q2_result.show()

+----------------+
|Number_of_Movies|
+----------------+
|            9742|
+----------------+



In [ ]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] <= 5)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by fewer than 5 users'.format(tmp1, tmp2))

6456 out of 9724 movies are rated by fewer than 5 users


In [ ]:
no_rating=spark.sql("Select Count(movieId) as Movies_Not_Rated From movies Where movieID not in (Select movieId From ratings)")
no_rating.show()

+----------------+
|Movies_Not_Rated|
+----------------+
|              18|
+----------------+



In [ ]:
movies_no_rating=spark.sql("Select movieId, title From movies Where movieID not in (Select movieId From ratings)")
movies_no_rating.toPandas()

,movieId,title
0,1076,"Innocents, The (1961)"
1,2939,Niagara (1953)
2,3338,For All Mankind (1989)
3,3456,"Color of Paradise, The (Rang-e khoda) (1999)"
4,4194,I Know Where I'm Going! (1945)
5,5721,"Chosen, The (1981)"
6,6668,"Road Home, The (Wo de fu qin mu qin) (1999)"
7,6849,Scrooge (1970)
8,7020,Proof (1991)
9,7792,"Parallax View, The (1974)"


**3. All movie genres**


In [ ]:
q3_result=spark.sql("Select Distinct explode(split(genres,'[|]')) as genres From movies Order by 1")
q3_result.show()

+------------------+
|            genres|
+------------------+
|(no genres listed)|
|            Action|
|         Adventure|
|         Animation|
|          Children|
|            Comedy|
|             Crime|
|       Documentary|
|             Drama|
|           Fantasy|
|         Film-Noir|
|            Horror|
|              IMAX|
|           Musical|
|           Mystery|
|           Romance|
|            Sci-Fi|
|          Thriller|
|               War|
|           Western|
+------------------+



**4. Number of movies in each genre**

In [ ]:
q4_result_1=spark.sql(
    "Select genres,Count(movieId) as Number_of_Movies From (Select explode(split(genres,'[|]')) as genres, movieId From movies) Group By 1 Order by 2 DESC")
q4_result_1.show()

+------------------+----------------+
|            genres|Number_of_Movies|
+------------------+----------------+
|             Drama|            4361|
|            Comedy|            3756|
|          Thriller|            1894|
|            Action|            1828|
|           Romance|            1596|
|         Adventure|            1263|
|             Crime|            1199|
|            Sci-Fi|             980|
|            Horror|             978|
|           Fantasy|             779|
|          Children|             664|
|         Animation|             611|
|           Mystery|             573|
|       Documentary|             440|
|               War|             382|
|           Musical|             334|
|           Western|             167|
|              IMAX|             158|
|         Film-Noir|              87|
|(no genres listed)|              34|
+------------------+----------------+



**5. Number of ratings**

In [ ]:
q5_result=spark.sql("Select Count(*) as Number_of_Ratings from ratings")
q5_result.show()

+-----------------+
|Number_of_Ratings|
+-----------------+
|           100836|
+-----------------+



**6. Number of movies in each tag, show only top 20 results**

In [ ]:
q6_result_1=spark.sql("Select tag, Count(movieId) as Number_of_Movies From tags Group By 1 Order by 2 DESC")
q6_result_1.show(20)

+------------------+----------------+
|               tag|Number_of_Movies|
+------------------+----------------+
|  In Netflix queue|             131|
|       atmospheric|              36|
| thought-provoking|              24|
|         superhero|              24|
|           surreal|              23|
|            Disney|              23|
|             funny|              23|
|          religion|              22|
|       dark comedy|              21|
|            quirky|              21|
|        psychology|              21|
|            sci-fi|              21|
|          suspense|              20|
|visually appealing|              19|
|      twist ending|              19|
|             crime|              19|
|          politics|              18|
|             music|              16|
|       time travel|              16|
|    mental illness|              16|
+------------------+----------------+
only showing top 20 rows



# **Part2: Spark ALS based approach for training model**

Using Spark ML to predict ratings with ratings.csv.

In [ ]:
ratings=ratings_df.drop('timestamp')

ratings = ratings.withColumn("userId", ratings["userId"].cast("Integer"))
ratings = ratings.withColumn("movieId", ratings["movieId"].cast("Integer"))
ratings = ratings.withColumn("rating", ratings["rating"].cast("float"))

In [ ]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)



# ALS Model Selection and Evaluation
With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [ ]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [ ]:
# split ratings to train and test sets: 0.8 to 0.2 ratio
(training,test)=ratings.randomSplit([0.8,0.2])

training.cache()
test.cache()

DataFrame[userId: int, movieId: int, rating: float]

In [ ]:
#Create ALS model
model = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [ ]:
#Tune model using ParamGridBuilder
paramGrid = (ParamGridBuilder()
             .addGrid(model.regParam, [0.05, 0.1, 0.3, 0.5])
             .addGrid(model.rank, [5, 10, 15])
             .addGrid(model.maxIter, [1, 5, 10])
             .build())

In [ ]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [ ]:
# Build Cross validation 
cv = CrossValidator(estimator=base, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [ ]:
#Fit ALS model to training data
cvModel = cv.fit(training)

In [ ]:
bestModel=cvModel.bestModel

# Model testing
And finally, make a prediction and check the testing error.

In [ ]:
#Generate predictions and evaluate using RMSE
predictions=bestModel.transform(test)
rmse = evaluator.evaluate(predictions)

In [ ]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank: ", str(bestModel._java_obj.parent().getRank())),
print (" MaxIter: ", str(bestModel._java_obj.parent().getMaxIter())), 
print (" RegParam: ", str(bestModel._java_obj.parent().getRegParam()))

RMSE = 1.081622125982638
**Best Model**
 Rank:  10
 MaxIter:  5
 RegParam:  0.01


In [ ]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   133|    471|   4.0|  3.745387|
|   436|    471|   3.0|   4.25896|
|   599|    471|   2.5| 4.0649853|
|   610|    471|   4.0| 3.5155776|
|   555|    471|   3.0|  3.691596|
|   176|    471|   5.0| 3.8968923|
|   448|    471|   4.0| 2.6877592|
|   608|    471|   1.5| 3.5397878|
|   541|    471|   3.0| 3.1751673|
|   260|    471|   4.5|  4.811924|
|   357|    471|   3.5| 2.8299525|
|   599|    833|   1.5| 1.1976092|
|   609|    833|   3.0| 1.1345911|
|   608|    833|   0.5| 1.9620548|
|   606|   1088|   3.0| 3.4504764|
|    64|   1088|   4.0| 3.2673013|
|   387|   1088|   1.5| 2.6032763|
|    51|   1088|   4.0| 3.0842583|
|   525|   1088|   4.5| 3.5194712|
|    42|   1088|   3.0| 3.9461255|
+------+-------+------+----------+
only showing top 20 rows



# Apply model on the entire dataset and analyze the performance

In [ ]:
alldata=bestModel.transform(ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6548876803732754


In [ ]:
alldata.registerTempTable("alldata")

In [ ]:
spark.sql("Select * From alldata").show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   191|    148|   5.0|  5.224857|
|   133|    471|   4.0|  3.745387|
|   597|    471|   2.0| 3.5172842|
|   385|    471|   4.0| 3.6759431|
|   436|    471|   3.0|   4.25896|
|   602|    471|   4.0| 3.7121518|
|    91|    471|   1.0| 2.1412864|
|   409|    471|   3.0|  2.892838|
|   372|    471|   3.0| 4.6815906|
|   599|    471|   2.5| 4.0649853|
|   603|    471|   4.0| 4.4310017|
|   182|    471|   4.5| 3.7322612|
|   218|    471|   4.0| 3.3882406|
|   474|    471|   3.0| 2.7381477|
|   500|    471|   1.0| 1.2007028|
|    57|    471|   3.0| 2.9827836|
|   462|    471|   2.5| 3.2799127|
|   387|    471|   3.0| 3.3425484|
|   610|    471|   4.0| 3.5155776|
|   217|    471|   2.0| 2.5376716|
+------+-------+------+----------+
only showing top 20 rows



In [ ]:
spark.sql("select * from movies join alldata on movies.movieId=alldata.movieId").show()

+-------+--------------------+------+------+-------+------+----------+
|movieId|               title|genres|userId|movieId|rating|prediction|
+-------+--------------------+------+------+-------+------+----------+
|    148|Awfully Big Adven...| Drama|   191|    148|   5.0|  5.224857|
|    471|Hudsucker Proxy, ...|Comedy|   133|    471|   4.0|  3.745387|
|    471|Hudsucker Proxy, ...|Comedy|   597|    471|   2.0| 3.5172842|
|    471|Hudsucker Proxy, ...|Comedy|   385|    471|   4.0| 3.6759431|
|    471|Hudsucker Proxy, ...|Comedy|   436|    471|   3.0|   4.25896|
|    471|Hudsucker Proxy, ...|Comedy|   602|    471|   4.0| 3.7121518|
|    471|Hudsucker Proxy, ...|Comedy|    91|    471|   1.0| 2.1412864|
|    471|Hudsucker Proxy, ...|Comedy|   409|    471|   3.0|  2.892838|
|    471|Hudsucker Proxy, ...|Comedy|   372|    471|   3.0| 4.6815906|
|    471|Hudsucker Proxy, ...|Comedy|   599|    471|   2.5| 4.0649853|
|    471|Hudsucker Proxy, ...|Comedy|   603|    471|   4.0| 4.4310017|
|    4

In [ ]:
print('Recommended top users (e.g. 1 top user) for all items with the corresponding predicted ratings:')
bestModel.recommendForAllItems(1).show(5)

print('-'*50)

print('Recommended top items (e.g. 1 top item) for all users with the corresponding predicted ratings:')
bestModel.recommendForAllUsers(1).show(5)

Recommended top users (e.g. 1 top user) for all items with the corresponding predicted ratings:
+-------+------------------+
|movieId|   recommendations|
+-------+------------------+
|   1580|  [[77, 6.348218]]|
|   4900|  [[598, 9.95858]]|
|   5300|[[257, 5.5355697]]|
|   6620|  [[99, 9.402206]]|
|   7340| [[407, 6.056511]]|
+-------+------------------+
only showing top 5 rows

--------------------------------------------------
Recommended top items (e.g. 1 top item) for all users with the corresponding predicted ratings:
+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|  [[1173, 8.669576]]|
|   463| [[2843, 6.5206275]]|
|   496|  [[3266, 9.939844]]|
|   148|[[119141, 7.27523...|
|   540| [[5992, 6.6425705]]|
+------+--------------------+
only showing top 5 rows



# Recommend movies to selected users (by userID)

In [ ]:
!pip install koalas


In [ ]:
pip install pyarrow==2.0.*

In [ ]:
import databricks.koalas as ks

In [ ]:
"""
recommendForAllUsers(int):

Returns top numItems items recommended for each user, for all users.
Parameters:
numItems - max number of recommendations for each user

"""

'\nrecommendForAllUsers(int):\n\nReturns top numItems items recommended for each user, for all users.\nParameters:\nnumItems - max number of recommendations for each user\n\n'

In [ ]:
itemRecs = bestModel.recommendForAllUsers(10)

In [ ]:
itemRecs.show(10)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[1173, 8.669576]...|
|   463|[[2843, 6.5206275...|
|   496|[[3266, 9.939844]...|
|   148|[[119141, 7.27523...|
|   540|[[5992, 6.6425705...|
|   392|[[720, 9.802321],...|
|   243|[[53127, 11.5328]...|
|    31|[[3441, 9.09947],...|
|   516|[[89904, 6.804111...|
|   580|[[1303, 6.52155],...|
+------+--------------------+
only showing top 10 rows



In [ ]:
itemRecs_ks=itemRecs.to_koalas()
movies_ks=movies_df.to_koalas()

In [ ]:
def movieRecommendation(inputId):
  recs_list=[]
  if inputId < 610: # make sure we can find that user id in our data 
    for recs in itemRecs_ks.loc[str(inputId), 'recommendations']:
      recs_list.append(str(recs[0]))
  return (movies_ks[movies_ks['movieId'].isin(recs_list)])

print("Recommended movies for selected user are as follows.")
movieRecommendation(int(input("enter user id: ")))

Recommended movies for selected user are as follows.
enter user id: 134


,movieId,title,genres
278,319,Shallow Grave (1994),Comedy|Drama|Thriller
720,940,"Adventures of Robin Hood, The (1938)",Action|Adventure|Romance
1692,2275,Six-String Samurai (1998),Action|Adventure|Sci-Fi
2277,3020,Falling Down (1993),Action|Drama
2684,3594,Center Stage (2000),Drama|Musical
2691,3608,Pee-wee's Big Adventure (1985),Adventure|Comedy
3240,4380,"Princess and the Warrior, The (Krieger und die...",Drama|Romance
4009,5666,"Rules of Attraction, The (2002)",Comedy|Drama|Romance|Thriller
4969,7579,Pride and Prejudice (1940),Comedy|Drama|Romance
5420,25850,Holiday (1938),Comedy|Drama|Romance


# Alternative way to recommend movies to a specific user

e.g: userId = 10

In [ ]:
# see historical rating of the user
user_history = training.filter(training['userId']==10)
user_history.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    10|    296|   1.0|
|    10|    588|   4.0|
|    10|    597|   3.5|
|    10|    912|   4.0|
|    10|   1088|   3.0|
|    10|   1247|   3.0|
|    10|   1307|   3.0|
|    10|   1784|   3.5|
|    10|   1907|   4.0|
|    10|   2671|   3.5|
|    10|   2762|   0.5|
|    10|   2959|   0.5|
|    10|   3578|   4.0|
|    10|   4246|   3.5|
|    10|   4306|   4.5|
|    10|   4447|   4.5|
|    10|   4993|   4.0|
|    10|   5066|   3.0|
|    10|   5377|   3.5|
|    10|   5620|   3.0|
+------+-------+------+
only showing top 20 rows



In [ ]:
# a list of movies we are thinking to offer
user_suggest = test.filter(training['userId']==10).select(['movieId', 'userId'])
user_suggest.show()

+-------+------+
|movieId|userId|
+-------+------+
|    356|    10|
|   1028|    10|
|   2571|    10|
|   2858|    10|
|   3882|    10|
|   4995|    10|
|   5952|    10|
|   7151|    10|
|   7153|    10|
|   7293|    10|
|   7375|    10|
|   7458|    10|
|   8533|    10|
|  41285|    10|
|  47099|    10|
|  49272|    10|
|  51662|    10|
|  51705|    10|
|  51834|    10|
|  56949|    10|
+-------+------+
only showing top 20 rows



In [ ]:
# offer movies with a high predicted rating
user_offer = bestModel.transform(user_suggest)
user_offer.orderBy('prediction', ascending=False).limit(10).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|   8533|    10|  5.078534|
|   2858|    10|  4.180396|
|  41285|    10| 4.0816617|
|   7458|    10| 4.0203485|
|  63992|    10| 3.9691901|
|  95543|    10| 3.9166398|
|  51705|    10| 3.9032626|
|  51662|    10| 3.7648196|
|   2571|    10| 3.7131803|
|  58047|    10| 3.6296153|
+-------+------+----------+



In [ ]:
user_offer.registerTempTable("user_offer")

In [ ]:
recs = spark.sql("select title from movies inner join user_offer on movies.movieId == user_offer.movieId")
recs.toPandas()

,title
0,Match Point (2005)
1,50 First Dates (2004)
2,Twilight (2008)
3,Becoming Jane (2007)
4,27 Dresses (2008)
5,Casino Royale (2006)
6,"Beautiful Mind, A (2001)"
7,Mary Poppins (1964)
8,Yes Man (2008)
9,"Matrix, The (1999)"


# Find the similar movies given a movie title

In [ ]:
itemFactors=bestModel.itemFactors.to_koalas()

In [ ]:
itemFactors.print_schema()

root
 |-- id: integer (nullable = false)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = false)



/usr/local/lib/python3.6/dist-packages/databricks/koalas/frame.py:5624: FutureWarning: DataFrame.print_schema is deprecated as of DataFrame.spark.print_schema. Please use the API instead.
  FutureWarning,


In [ ]:
# merging itemFactors and movies_fs
# we can ask user to enter a movie name and find its movie id, 
# which is used in a function called similarMovies to find its top 10 similar movies.
# It's easier for an user to know the movie name than knowing movieId, thus we want the user to search by movie name.

merge_kdf = movies_ks.merge(itemFactors, left_on='movieId', right_on='id')

In [ ]:
merge_kdf.head()

,movieId,title,genres,id,features
0,10,GoldenEye (1995),Action|Adventure|Thriller,10,"[1.2526657581329346, -2.8258423805236816, 0.77..."
1,20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller,20,"[0.4498746395111084, -2.6670618057250977, 1.12..."
2,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama,30,"[0.6992306113243103, -2.93139910697937, 1.3934..."
3,40,"Cry, the Beloved Country (1995)",Drama,40,"[1.8337295055389404, 0.16487917304039001, -0.8..."
4,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,50,"[1.3172078132629395, -3.900089740753174, 1.348..."


In [ ]:
merge_kdf.print_schema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: integer (nullable = false)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = false)



/usr/local/lib/python3.6/dist-packages/databricks/koalas/frame.py:5624: FutureWarning: DataFrame.print_schema is deprecated as of DataFrame.spark.print_schema. Please use the API instead.
  FutureWarning,


In [ ]:
# helper function to get movieId when user searches by movie name

def _get_mov_id():
  title = input("Enter movie name: ")
  return merge_kdf[merge_kdf.title==title].movieId.values[0]

In [ ]:
_get_mov_id()

Enter movie name: Usual Suspects, The (1995)


/usr/local/lib/python3.6/dist-packages/databricks/koalas/generic.py:567: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))


'50'

In [ ]:
# now use the movie id we got above to get top 10 similar movies

def similarMovies(inputId, matrix='cosine_similarity'):
  try:
    movieFeature=merge_kdf.loc[merge_kdf.id==str(inputId),'features'].to_numpy()[0]
  except:
    return 'There is no movie with id ' + str(inputId)
  
  if matrix=='cosine_similarity':
    similarMovie=pd.DataFrame(columns=('movieId','cosine_similarity'))

    for id,feature in itemFactors.to_numpy():
      cs=np.dot(movieFeature,feature)/(np.linalg.norm(movieFeature) * np.linalg.norm(feature))
      similarMovie=similarMovie.append({'movieId':str(id), 'cosine_similarity':cs}, ignore_index=True)
    
    similarMovie_cs=similarMovie.sort_values(by=['cosine_similarity'],ascending = False)
    
  return merge_kdf[['movieId','title','genres']].head(10)

In [ ]:
similarMovies(50)

,movieId,title,genres
0,10,GoldenEye (1995),Action|Adventure|Thriller
1,20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller
2,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama
3,40,"Cry, the Beloved Country (1995)",Drama
4,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,60,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy
6,70,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller
7,80,"White Balloon, The (Badkonake sefid) (1995)",Children|Drama
8,100,City Hall (1996),Drama|Thriller
9,110,Braveheart (1995),Action|Drama|War
